In [4]:
from sympy import Symbol, sympify, Function, Expr
from sympy.parsing.latex import parse_latex
from sympy.core.function import AppliedUndef
from ExpDerive.derive import expression
from tests.resolver import resolver, eval_resolver, func_resolver



In [20]:
def f1 (x, y):
    return x + y
f1.__code__.co_varnames[:f1.__code__.co_argcount]
# F = resolve.Func(func=f1)
# F2 = resolve.Func(latex='h\\cdot\\frac{f}{3}')
# F2(1, 2)


('x', 'y')

In [24]:
f = Function('g')('x', 'y')

In [22]:
ex = parse_latex('f\left(x\cdot y \\right) + f\left(3\cdot z \\right)')

func_names = set(map(
    lambda f: f.func.__name__,
    ex.atoms(AppliedUndef)
))
func_names

{'f'}

In [28]:
for s in ex.atoms(AppliedUndef):
    print(ex.subs(s, f))

f(3*z) + g(x, y)
f(x*y) + g(x, y)


In [12]:
import inspect
inspect.signature(f1)

mappingproxy({'x': <Parameter "x">, 'y': <Parameter "y">})

In [16]:

def test_build_tree():
    tree = resolve.ExpTree('x+\\frac{y}{2}', 'g', resolver)
    tree.build_expression()
    print(tree.tree.expression)

l = 'f\left(x-2\\right)\cdot \\frac{\mathit{ast}}{3}'
# l = 'x+\\frac{y}{2}'
tree = resolve.ExpTree(l, 'g', resolver)
tree.func_resolver = func_resolver
tree.build_expression()

(ast/3)*f(x - 1*2)
{h(x)} 11 h(x) - 1*3
set() 11 x**2
x**2 22
x**2 - 3 22
[(x, x - 1*2)] f(x - 1*2) x**2 - 3
(x - 1*2)**2 - 3
ast*((x - 1*2)**2 - 3)/3
ast*2
ast*2
ast
ast*((x - 1*2)**2 - 3)/3
ast
f\left(x-2\right)\cdot \frac{\mathit{ast}}{3}
\mathit{ast}*2
ast*2 ast <ExpDerive.derive.resolve.Var object at 0x125ca4370>
//////////
ast*((x - 1*2)**2 - 3)/3 x <ExpDerive.derive.resolve.Var object at 0x125ceff70>
ast*((ast*2 - 2)**2 - 3)/3 ast <ExpDerive.derive.resolve.Var object at 0x125cdb8e0>
//////////


In [18]:
n = tree.tree.expression
print(n.evalf(subs={'ast': 7}), n.evalf(), n)

329.000000000000 0.333333333333333*ast*(4.0*(ast - 1)**2 - 3.0) ast*((ast*2 - 2)**2 - 3)/3


In [ ]:
print(tree.tree.expression)
f = Function('f')(*[Symbol('x')])
e = f + 1
print(e.evalf(subs={'x': 2}), e.atoms(AppliedUndef))
# Symbol('x').atoms(AppliedUndef)

In [ ]:
exp = parse_latex('f\left(s\\right)+2')
fun = parse_latex('s^2')
exp.atoms(AppliedUndef)
for f in exp.atoms(AppliedUndef):
    print(f.args)
    print(exp.subs(f, fun), exp)

In [ ]:
subjects = ['pl1', 'pl2', 'pl3']
tree.eval_resolver = eval_resolver
vals = tree.evaluate(subjects)
print(vals)

In [ ]:
# latex is sum of a and b over 2
latex = "f\left(a"+"\\right)-b"
latex = "\cos \left(b\\right)"
print(latex)
# for v in ['a', 'b']:
#     latex = latex.replace(v, '\mathit{'+v+'}')
#     print(latex)
exp = parse_latex(latex)
print(str(exp))


In [ ]:
vars = exp.atoms(Symbol)
{
    str(v): 1
    for v in vars
}

In [ ]:
print(g="g")